In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, recall_score
import joblib

# Load dataset
DATA_PATH = r"C:\Users\moham\Downloads\English_Words\hand_landmarks_data.csv"
df = pd.read_csv(DATA_PATH)

# Preprocessing: Normalize landmarks relative to wrist and scale by middle finger tip
def preprocess_data(df):
    df_scaled = df.copy()
    
    # Center landmarks around wrist (joint 1)
    wrist_x = df_scaled['x1'].copy()
    wrist_y = df_scaled['y1'].copy()
    
    for i in range(1, 22):
        df_scaled[f'x{i}'] -= wrist_x
        df_scaled[f'y{i}'] -= wrist_y
    
    # Normalize by middle finger tip distance (joint 13)
    mid_finger_tip_dist = np.sqrt(df_scaled['x13']**2 + df_scaled['y13']**2)
    
    for i in range(1, 22):
        df_scaled[f'x{i}'] /= mid_finger_tip_dist
        df_scaled[f'y{i}'] /= mid_finger_tip_dist
    
    return df_scaled

df_scaled = preprocess_data(df)

# Split features and labels
X = df_scaled.drop("label", axis=1)
y = df_scaled["label"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=100
)

# SVM with GridSearch
param_grid = {
    'kernel': ['rbf'],
    'C': [0.01, 0.1, 1, 10, 100],
    'gamma': [0.01, 0.1, 1, 10, 100]
}

svm = SVC(class_weight='balanced')
grid_search = GridSearchCV(svm, param_grid, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get best model
best_model = grid_search.best_estimator_

# Save model and label classes
joblib.dump({
    'model': best_model,
    'label_classes': best_model.classes_
}, "svm_model_with_labels.pkl")

# Optional: Print evaluation metrics
y_pred = best_model.predict(X_test)
print(f"Accuracy: {best_model.score(X_test, y_test):.2f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted'):.2f}")

# Load model and labels
saved_data = joblib.load("svm_model_with_labels.pkl")
model = saved_data['model']
label_classes = saved_data['label_classes']


Accuracy: 0.99
F1 Score: 0.99
